In [27]:
pose_landmark_labels = {
    0: "NOSE",
    1: "LEFT_EYE_INNER",
    2: "LEFT_EYE",
    3: "LEFT_EYE_OUTER",
    4: "RIGHT_EYE_INNER",
    5: "RIGHT_EYE",
    6: "RIGHT_EYE_OUTER",
    7: "LEFT_EAR",
    8: "RIGHT_EAR",
    9: "MOUTH_LEFT",
    10: "MOUTH_RIGHT",
    11: "LEFT_SHOULDER",   
    12: "RIGHT_SHOULDER",  
    13: "LEFT_ELBOW",     
    14: "RIGHT_ELBOW",     
    15: "LEFT_WRIST",      
    16: "RIGHT_WRIST",     
    17: "LEFT_PINKY",      
    18: "RIGHT_PINKY",     
    19: "LEFT_INDEX",      
    20: "RIGHT_INDEX",     
    21: "LEFT_THUMB",      
    22: "RIGHT_THUMB",    
    23: "LEFT_HIP",        
    24: "RIGHT_HIP",       
    25: "LEFT_KNEE",       
    26: "RIGHT_KNEE",      
    27: "LEFT_ANKLE",      
    28: "RIGHT_ANKLE",     
    29: "LEFT_HEEL",       
    30: "RIGHT_HEEL",      
    31: "LEFT_FOOT_INDEX",
    32: "RIGHT_FOOT_INDEX" 
}

In [28]:
face_neck = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10] # Mặt -> Cổ
right_arm = [12, 14, 16, 18, 20, 22]  # Vai phải -> Khuỷu phải -> Cổ tay phải -> Ngón út phải -> Ngón trỏ phải -> Ngón cái phải
left_arm = [11, 13, 15, 17, 19, 21]   # Vai trái -> Khuỷu trái -> Cổ tay trái -> Ngón út trái -> Ngón trỏ trái -> Ngón cái trái
right_leg = [24, 26, 28, 30, 32]      # Hông phải -> Đầu gối phải -> Mắt cá phải -> Gót chân phải -> Ngón chân phải
left_leg = [23, 25, 27, 29, 31]       # Hông trái -> Đầu gối trái -> Mắt cá trái -> Gót chân trái -> Ngón chân trái
body = [11, 12, 23, 24]                # Vai trái -> Vai phải -> Hông trái -> Hông phải

In [23]:
import os
import shutil
import csv
import cv2
import mediapipe as mp

# Khởi tạo MediaPipe Pose
mp_pose = mp.solutions.pose
pose = mp_pose.Pose(static_image_mode=True)

# Đọc tất cả ảnh trong thư mục
folder_path = "output_data1/processed_keyframes"  # Đường dẫn thư mục chứa ảnh
image_files = [f for f in os.listdir(folder_path) if f.endswith(".jpg")]

# Định nghĩa các nhóm keypoint
KEYPOINT_GROUPS = {
    "FACE_NECK": [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10],
    "RIGHT_ARM": [12, 14, 16, 18, 20, 22],
    "LEFT_ARM": [11, 13, 15, 17, 19, 21],
    "RIGHT_LEG": [24, 26, 28, 30, 32],
    "LEFT_LEG": [23, 25, 27, 29, 31],
    "BODY": [11, 12, 23, 24]
}

# Hàm trích xuất tọa độ x, y, z từ nhóm keypoint
def extract_keypoint_coordinates(results, group_ids, width, height):
    coordinates = []
    for landmark_id in group_ids:
        landmark = results.pose_landmarks.landmark[landmark_id]
        x = landmark.x * width
        y = landmark.y * height
        z = landmark.z  # Độ sâu
        coordinates.extend([x, y, z])  # Lưu x, y, z vào danh sách
    return coordinates

# Khởi tạo danh sách cho các bộ phận cơ thể
coordinates_by_group = {group: [] for group in KEYPOINT_GROUPS}

# Xử lý tất cả các ảnh trong thư mục
for image_file in image_files:
    # Đọc ảnh và xử lý
    image_path = os.path.join(folder_path, image_file)
    image = cv2.imread(image_path)
    image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    results = pose.process(image_rgb)

    # Lấy kích thước ảnh
    height, width, _ = image.shape

    # Nếu phát hiện được các keypoints
    if results.pose_landmarks:
        for group_name, ids in KEYPOINT_GROUPS.items():
            coordinates = extract_keypoint_coordinates(results, ids, width, height)
            coordinates_by_group[group_name].append(coordinates)

# Tạo thư mục để lưu các file CSV nếu chưa tồn tại
output_folder = "pose_data_csv1"
if not os.path.exists(output_folder):
    os.makedirs(output_folder)

# Lưu dữ liệu vào các file CSV riêng biệt cho mỗi bộ phận cơ thể
for group_name, coordinates_list in coordinates_by_group.items():
    file_path = os.path.join(output_folder, f"{group_name}_coordinates.csv")
    with open(file_path, "w", newline="") as csvfile:
        writer = csv.writer(csvfile)
        
        # Tạo các tiêu đề cột
        header = []
        for i in range(1, len(KEYPOINT_GROUPS[group_name]) + 1):
            header.extend([f"{group_name}_{i}_x", f"{group_name}_{i}_y", f"{group_name}_{i}_z"])
        writer.writerow(header)
        
        # Viết các tọa độ vào file
        for coordinates in coordinates_list:
            writer.writerow(coordinates)

    print(f"Đã lưu dữ liệu của {group_name} vào file {file_path}")


Đã lưu dữ liệu của FACE_NECK vào file pose_data_csv\FACE_NECK_coordinates.csv
Đã lưu dữ liệu của RIGHT_ARM vào file pose_data_csv\RIGHT_ARM_coordinates.csv
Đã lưu dữ liệu của LEFT_ARM vào file pose_data_csv\LEFT_ARM_coordinates.csv
Đã lưu dữ liệu của RIGHT_LEG vào file pose_data_csv\RIGHT_LEG_coordinates.csv
Đã lưu dữ liệu của LEFT_LEG vào file pose_data_csv\LEFT_LEG_coordinates.csv
Đã lưu dữ liệu của BODY vào file pose_data_csv\BODY_coordinates.csv


In [3]:
import os
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [51]:
# bây giờ có 2 folder chứa dữ liệu pose, ta sẽ kết hợp chúng lại theo từng file csv bên trong
# BODY của data1 và data2 sẽ được kết hợp lại với nhau thành 1 file csv mới
# tương tự từng bộ phận còn lại sẽ được kết hợp với nhau thành 1 file csv mới

# đọc dữ liệu 2 file BODY_coordinates.csv từ 2 folder pose_data_csv1 và pose_data_csv2
df1 = pd.read_csv("pose_data_csv1/BODY_coordinates.csv")
df2 = pd.read_csv("pose_data_csv2/BODY_coordinates.csv")

# kết hợp 2 file BODY_coordinates.csv lại với nhau
df = pd.concat([df1, df2], axis=0)

# lưu vào folder mới là pose_data_csv
# tạo folder pose_data_csv nếu chưa tồn tại
if not os.path.exists("pose_data_csv"):
    os.makedirs("pose_data_csv")
    
# lưu file BODY_coordinates.csv vào folder pose_data_csv
df.to_csv("pose_data_csv/BODY_coordinates.csv", index=False)

In [52]:
# đọc dữ liệu 2 file FACE_NECK_coordinates.csv từ 2 folder pose_data_csv1 và pose_data_csv2
df1 = pd.read_csv("pose_data_csv1/FACE_NECK_coordinates.csv")
df2 = pd.read_csv("pose_data_csv2/FACE_NECK_coordinates.csv")

# kết hợp 2 file FACE_NECK_coordinates.csv lại với nhau
df = pd.concat([df1, df2], axis=0)

# lưu vào folder mới là pose_data_csv
df.to_csv("pose_data_csv/FACE_NECK_coordinates.csv", index=False)

In [53]:
# đọc dữ liệu 2 file RIGHT_ARM_coordinates.csv từ 2 folder pose_data_csv1 và pose_data_csv2
df1 = pd.read_csv("pose_data_csv1/RIGHT_ARM_coordinates.csv")
df2 = pd.read_csv("pose_data_csv2/RIGHT_ARM_coordinates.csv")

# kết hợp 2 file RIGHT_ARM_coordinates.csv lại với nhau
df = pd.concat([df1, df2], axis=0)

# lưu vào folder mới là pose_data_csv
df.to_csv("pose_data_csv/RIGHT_ARM_coordinates.csv", index=False)

In [54]:
# đọc dữ liệu 2 file LEFT_ARM_coordinates.csv từ 2 folder pose_data_csv1 và pose_data_csv2
df1 = pd.read_csv("pose_data_csv1/LEFT_ARM_coordinates.csv")
df2 = pd.read_csv("pose_data_csv2/LEFT_ARM_coordinates.csv")

# kết hợp 2 file LEFT_ARM_coordinates.csv lại với nhau
df = pd.concat([df1, df2], axis=0)

# lưu vào folder mới là pose_data_csv   
df.to_csv("pose_data_csv/LEFT_ARM_coordinates.csv", index=False)

In [55]:
# đọc dữ liệu 2 file RIGHT_LEG_coordinates.csv từ 2 folder pose_data_csv1 và pose_data_csv2
df1 = pd.read_csv("pose_data_csv1/RIGHT_LEG_coordinates.csv")
df2 = pd.read_csv("pose_data_csv2/RIGHT_LEG_coordinates.csv")

# kết hợp 2 file RIGHT_LEG_coordinates.csv lại với nhau
df = pd.concat([df1, df2], axis=0)

# lưu vào folder mới là pose_data_csv
df.to_csv("pose_data_csv/RIGHT_LEG_coordinates.csv", index=False)

In [56]:
# đọc dữ liệu 2 file LEFT_LEG_coordinates.csv từ 2 folder pose_data_csv1 và pose_data_csv2
df1 = pd.read_csv("pose_data_csv1/LEFT_LEG_coordinates.csv")
df2 = pd.read_csv("pose_data_csv2/LEFT_LEG_coordinates.csv")

# kết hợp 2 file LEFT_LEG_coordinates.csv lại với nhau
df = pd.concat([df1, df2], axis=0)

# lưu vào folder mới là pose_data_csv
df.to_csv("pose_data_csv/LEFT_LEG_coordinates.csv", index=False)

In [59]:
# thêm label cho file csv mới tạo, label sẽ được lưu trong cột "label"
# label sẽ được gán giá trị 1 cho data1 và 0 cho data2 (127 dòng đầu tiên là data1, 162 dòng cuối cùng là data2)

# body_coordinates.csv
df1 = pd.read_csv("pose_data_csv/BODY_coordinates.csv")
df1["label"] = [1] * 127 + [0] * 162
df1.to_csv("pose_data_csv/BODY_coordinates.csv", index=False)

# face_neck_coordinates.csv
df2 = pd.read_csv("pose_data_csv/FACE_NECK_coordinates.csv")
df2["label"] = [1] * 127 + [0] * 162
df2.to_csv("pose_data_csv/FACE_NECK_coordinates.csv", index=False)

# right_arm_coordinates.csv
df3 = pd.read_csv("pose_data_csv/RIGHT_ARM_coordinates.csv")
df3["label"] = [1] * 127 + [0] * 162
df3.to_csv("pose_data_csv/RIGHT_ARM_coordinates.csv", index=False)

# left_arm_coordinates.csv
df4 = pd.read_csv("pose_data_csv/LEFT_ARM_coordinates.csv")
df4["label"] = [1] * 127 + [0] * 162
df4.to_csv("pose_data_csv/LEFT_ARM_coordinates.csv", index=False)

# right_leg_coordinates.csv
df5 = pd.read_csv("pose_data_csv/RIGHT_LEG_coordinates.csv")
df5["label"] = [1] * 127 + [0] * 162
df5.to_csv("pose_data_csv/RIGHT_LEG_coordinates.csv", index=False)

# left_leg_coordinates.csv
df6 = pd.read_csv("pose_data_csv/LEFT_LEG_coordinates.csv")
df6["label"] = [1] * 127 + [0] * 162
df6.to_csv("pose_data_csv/LEFT_LEG_coordinates.csv", index=False)

In [64]:
# tạo dữ liệu huấn luyện và kiểm tra 
# đọc dữ liệu từ các file csv
df1 = pd.read_csv("pose_data_csv/BODY_coordinates.csv")
df2 = pd.read_csv("pose_data_csv/FACE_NECK_coordinates.csv")
df3 = pd.read_csv("pose_data_csv/RIGHT_ARM_coordinates.csv")
df4 = pd.read_csv("pose_data_csv/LEFT_ARM_coordinates.csv")
df5 = pd.read_csv("pose_data_csv/RIGHT_LEG_coordinates.csv")
df6 = pd.read_csv("pose_data_csv/LEFT_LEG_coordinates.csv")

# chia dữ liệu thành dữ liệu huấn luyện và kiểm tra và lưu vào 6 folder riêng biệt
# tạo folder train và test nếu chưa tồn tại
# folder cho BODY
if not os.path.exists("pose_data_csv/train/BODY"):
    os.makedirs("pose_data_csv/BODY/train")
    os.makedirs("pose_data_csv/BODY/test")

# folder cho FACE_NECK
if not os.path.exists("pose_data_csv/train/FACE_NECK"):
    os.makedirs("pose_data_csv/FACE_NECK/train")
    os.makedirs("pose_data_csv/FACE_NECK/test")
    
# folder cho RIGHT_ARM
if not os.path.exists("pose_data_csv/train/RIGHT_ARM"):
    os.makedirs("pose_data_csv/RIGHT_ARM/train")
    os.makedirs("pose_data_csv/RIGHT_ARM/test")
    
# folder cho LEFT_ARM
if not os.path.exists("pose_data_csv/train/LEFT_ARM"):
    os.makedirs("pose_data_csv/LEFT_ARM/train")
    os.makedirs("pose_data_csv/LEFT_ARM/test")
    
# folder cho RIGHT_LEG
if not os.path.exists("pose_data_csv/train/RIGHT_LEG"):
    os.makedirs("pose_data_csv/RIGHT_LEG/train")
    os.makedirs("pose_data_csv/RIGHT_LEG/test")
    
# folder cho LEFT_LEG
if not os.path.exists("pose_data_csv/train/LEFT_LEG"):
    os.makedirs("pose_data_csv/LEFT_LEG/train")
    os.makedirs("pose_data_csv/LEFT_LEG/test")

In [65]:
# chia dữ liệu thành dữ liệu huấn luyện và kiểm tra
# BODY
X1 = df1.drop("label", axis=1)
y1 = df1["label"]
X_train1, X_test1, y_train1, y_test1 = train_test_split(X1, y1, test_size=0.2, random_state=42)
X_train1.to_csv("pose_data_csv/BODY/train/X_train.csv", index=False)
X_test1.to_csv("pose_data_csv/BODY/test/X_test.csv", index=False)
y_train1.to_csv("pose_data_csv/BODY/train/y_train.csv", index=False)
y_test1.to_csv("pose_data_csv/BODY/test/y_test.csv", index=False)

# FACE_NECK
X2 = df2.drop("label", axis=1)
y2 = df2["label"]
X_train2, X_test2, y_train2, y_test2 = train_test_split(X2, y2, test_size=0.2, random_state=42)
X_train2.to_csv("pose_data_csv/FACE_NECK/train/X_train.csv", index=False)
X_test2.to_csv("pose_data_csv/FACE_NECK/test/X_test.csv", index=False)
y_train2.to_csv("pose_data_csv/FACE_NECK/train/y_train.csv", index=False)
y_test2.to_csv("pose_data_csv/FACE_NECK/test/y_test.csv", index=False)

# RIGHT_ARM
X3 = df3.drop("label", axis=1)
y3 = df3["label"]
X_train3, X_test3, y_train3, y_test3 = train_test_split(X3, y3, test_size=0.2, random_state=42)
X_train3.to_csv("pose_data_csv/RIGHT_ARM/train/X_train.csv", index=False)
X_test3.to_csv("pose_data_csv/RIGHT_ARM/test/X_test.csv", index=False)
y_train3.to_csv("pose_data_csv/RIGHT_ARM/train/y_train.csv", index=False)
y_test3.to_csv("pose_data_csv/RIGHT_ARM/test/y_test.csv", index=False)

# LEFT_ARM
X4 = df4.drop("label", axis=1)
y4 = df4["label"]
X_train4, X_test4, y_train4, y_test4 = train_test_split(X4, y4, test_size=0.2, random_state=42)
X_train4.to_csv("pose_data_csv/LEFT_ARM/train/X_train.csv", index=False)
X_test4.to_csv("pose_data_csv/LEFT_ARM/test/X_test.csv", index=False)
y_train4.to_csv("pose_data_csv/LEFT_ARM/train/y_train.csv", index=False)
y_test4.to_csv("pose_data_csv/LEFT_ARM/test/y_test.csv", index=False)

# RIGHT_LEG
X5 = df5.drop("label", axis=1)
y5 = df5["label"]
X_train5, X_test5, y_train5, y_test5 = train_test_split(X5, y5, test_size=0.2, random_state=42)
X_train5.to_csv("pose_data_csv/RIGHT_LEG/train/X_train.csv", index=False)
X_test5.to_csv("pose_data_csv/RIGHT_LEG/test/X_test.csv", index=False)
y_train5.to_csv("pose_data_csv/RIGHT_LEG/train/y_train.csv", index=False)
y_test5.to_csv("pose_data_csv/RIGHT_LEG/test/y_test.csv", index=False)

# LEFT_LEG
X6 = df6.drop("label", axis=1)
y6 = df6["label"]
X_train6, X_test6, y_train6, y_test6 = train_test_split(X6, y6, test_size=0.2, random_state=42)
X_train6.to_csv("pose_data_csv/LEFT_LEG/train/X_train.csv", index=False)
X_test6.to_csv("pose_data_csv/LEFT_LEG/test/X_test.csv", index=False)
y_train6.to_csv("pose_data_csv/LEFT_LEG/train/y_train.csv", index=False)
y_test6.to_csv("pose_data_csv/LEFT_LEG/test/y_test.csv", index=False)

In [122]:
from sklearn.preprocessing import MinMaxScaler

# chuẩn hóa dữ liệu
# BODY
scaler1 = MinMaxScaler()
X_train1 = pd.read_csv("pose_data_csv/BODY/train/X_train.csv")
X_test1 = pd.read_csv("pose_data_csv/BODY/test/X_test.csv")
scaler1.fit(X_train1)
X_train1 = scaler1.transform(X_train1)
X_test1 = scaler1.transform(X_test1)
pd.DataFrame(X_train1).to_csv("pose_data_csv/BODY/train/X_train.csv", index=False)
pd.DataFrame(X_test1).to_csv("pose_data_csv/BODY/test/X_test.csv", index=False)

# FACE_NECK
scaler2 = MinMaxScaler()
X_train2 = pd.read_csv("pose_data_csv/FACE_NECK/train/X_train.csv")
X_test2 = pd.read_csv("pose_data_csv/FACE_NECK/test/X_test.csv")
scaler2.fit(X_train2)
X_train2 = scaler2.transform(X_train2)
X_test2 = scaler2.transform(X_test2)
pd.DataFrame(X_train2).to_csv("pose_data_csv/FACE_NECK/train/X_train.csv", index=False)
pd.DataFrame(X_test2).to_csv("pose_data_csv/FACE_NECK/test/X_test.csv", index=False)

# RIGHT_ARM
scaler3 = MinMaxScaler()
X_train3 = pd.read_csv("pose_data_csv/RIGHT_ARM/train/X_train.csv")
X_test3 = pd.read_csv("pose_data_csv/RIGHT_ARM/test/X_test.csv")
scaler3.fit(X_train3)
X_train3 = scaler3.transform(X_train3)
X_test3 = scaler3.transform(X_test3)
pd.DataFrame(X_train3).to_csv("pose_data_csv/RIGHT_ARM/train/X_train.csv", index=False)
pd.DataFrame(X_test3).to_csv("pose_data_csv/RIGHT_ARM/test/X_test.csv", index=False)

# LEFT_ARM
scaler4 = MinMaxScaler()
X_train4 = pd.read_csv("pose_data_csv/LEFT_ARM/train/X_train.csv")
X_test4 = pd.read_csv("pose_data_csv/LEFT_ARM/test/X_test.csv")
scaler4.fit(X_train4)
X_train4 = scaler4.transform(X_train4)
X_test4 = scaler4.transform(X_test4)
pd.DataFrame(X_train4).to_csv("pose_data_csv/LEFT_ARM/train/X_train.csv", index=False)
pd.DataFrame(X_test4).to_csv("pose_data_csv/LEFT_ARM/test/X_test.csv", index=False)

# RIGHT_LEG
scaler5 = MinMaxScaler()
X_train5 = pd.read_csv("pose_data_csv/RIGHT_LEG/train/X_train.csv")
X_test5 = pd.read_csv("pose_data_csv/RIGHT_LEG/test/X_test.csv")
scaler5.fit(X_train5)
X_train5 = scaler5.transform(X_train5)
X_test5 = scaler5.transform(X_test5)
pd.DataFrame(X_train5).to_csv("pose_data_csv/RIGHT_LEG/train/X_train.csv", index=False)
pd.DataFrame(X_test5).to_csv("pose_data_csv/RIGHT_LEG/test/X_test.csv", index=False)

# LEFT_LEG
scaler6 = MinMaxScaler()
X_train6 = pd.read_csv("pose_data_csv/LEFT_LEG/train/X_train.csv")
X_test6 = pd.read_csv("pose_data_csv/LEFT_LEG/test/X_test.csv")
scaler6.fit(X_train6)
X_train6 = scaler6.transform(X_train6)
X_test6 = scaler6.transform(X_test6)
pd.DataFrame(X_train6).to_csv("pose_data_csv/LEFT_LEG/train/X_train.csv", index=False)
pd.DataFrame(X_test6).to_csv("pose_data_csv/LEFT_LEG/test/X_test.csv", index=False)

In [154]:
# tạo mô hình học máy, huấn luyện và đánh giá mô hình, sử dụng nhiều mô hình khác nhau để so sánh kết quả
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

# BODY
X_train1 = pd.read_csv("pose_data_csv/BODY/train/X_train.csv").values
y_train1 = pd.read_csv("pose_data_csv/BODY/train/y_train.csv").values
X_test1 = pd.read_csv("pose_data_csv/BODY/test/X_test.csv").values
y_test1 = pd.read_csv("pose_data_csv/BODY/test/y_test.csv").values


# Logistic Regression
model1 = LogisticRegression()
model1.fit(X_train1, y_train1)
y_pred1 = model1.predict(X_test1)
accuracy1 = accuracy_score(y_test1, y_pred1)
print(f"BODY - Logistic Regression: {accuracy1}")

# Support Vector Machine
model2 = SVC()
model2.fit(X_train1, y_train1)
y_pred2 = model2.predict(X_test1)
accuracy2 = accuracy_score(y_test1, y_pred2)
print(f"BODY - Support Vector Machine: {accuracy2}")

# Random Forest
model3 = RandomForestClassifier()
model3.fit(X_train1, y_train1)
y_pred3 = model3.predict(X_test1)
accuracy3 = accuracy_score(y_test1, y_pred3)
print(f"BODY - Random Forest: {accuracy3}")

BODY - Logistic Regression: 1.0
BODY - Support Vector Machine: 1.0
BODY - Random Forest: 1.0


d:\enviroments\envs\pytorch\lib\site-packages\sklearn\utils\validation.py:1408: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
d:\enviroments\envs\pytorch\lib\site-packages\sklearn\utils\validation.py:1408: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
d:\enviroments\envs\pytorch\lib\site-packages\sklearn\base.py:1389: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)


In [157]:
# FACE_NECK
X_train2 = pd.read_csv("pose_data_csv/FACE_NECK/train/X_train.csv").values
y_train2 = pd.read_csv("pose_data_csv/FACE_NECK/train/y_train.csv").values
X_test2 = pd.read_csv("pose_data_csv/FACE_NECK/test/X_test.csv").values
y_test2 = pd.read_csv("pose_data_csv/FACE_NECK/test/y_test.csv").values

# Logistic Regression
model4 = LogisticRegression()
model4.fit(X_train2, y_train2)
y_pred4 = model4.predict(X_test2)
accuracy4 = accuracy_score(y_test2, y_pred4)
print(f"FACE_NECK - Logistic Regression: {accuracy4}")

# Support Vector Machine
model5 = SVC()
model5.fit(X_train2, y_train2)
y_pred5 = model5.predict(X_test2)
accuracy5 = accuracy_score(y_test2, y_pred5)
print(f"FACE_NECK - Support Vector Machine: {accuracy5}")

# Random Forest
model6 = RandomForestClassifier()
model6.fit(X_train2, y_train2)
y_pred6 = model6.predict(X_test2)
accuracy6 = accuracy_score(y_test2, y_pred6)
print(f"FACE_NECK - Random Forest: {accuracy6}")

FACE_NECK - Logistic Regression: 1.0
FACE_NECK - Support Vector Machine: 1.0
FACE_NECK - Random Forest: 1.0


d:\enviroments\envs\pytorch\lib\site-packages\sklearn\utils\validation.py:1408: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
d:\enviroments\envs\pytorch\lib\site-packages\sklearn\utils\validation.py:1408: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
d:\enviroments\envs\pytorch\lib\site-packages\sklearn\base.py:1389: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)


In [159]:
# RIGHT_ARM
X_train3 = pd.read_csv("pose_data_csv/RIGHT_ARM/train/X_train.csv").values
y_train3 = pd.read_csv("pose_data_csv/RIGHT_ARM/train/y_train.csv").values
X_test3 = pd.read_csv("pose_data_csv/RIGHT_ARM/test/X_test.csv").values
y_test3 = pd.read_csv("pose_data_csv/RIGHT_ARM/test/y_test.csv").values

# Logistic Regression
model7 = LogisticRegression()
model7.fit(X_train3, y_train3)
y_pred7 = model7.predict(X_test3)
accuracy7 = accuracy_score(y_test3, y_pred7)
print(f"RIGHT_ARM - Logistic Regression: {accuracy7}")

# Support Vector Machine
model8 = SVC()
model8.fit(X_train3, y_train3)
y_pred8 = model8.predict(X_test3)
accuracy8 = accuracy_score(y_test3, y_pred8)
print(f"RIGHT_ARM - Support Vector Machine: {accuracy8}")

# Random Forest
model9 = RandomForestClassifier()
model9.fit(X_train3, y_train3)
y_pred9 = model9.predict(X_test3)
accuracy9 = accuracy_score(y_test3, y_pred9)
print(f"RIGHT_ARM - Random Forest: {accuracy9}")

RIGHT_ARM - Logistic Regression: 1.0
RIGHT_ARM - Support Vector Machine: 1.0
RIGHT_ARM - Random Forest: 1.0


d:\enviroments\envs\pytorch\lib\site-packages\sklearn\utils\validation.py:1408: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
d:\enviroments\envs\pytorch\lib\site-packages\sklearn\utils\validation.py:1408: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
d:\enviroments\envs\pytorch\lib\site-packages\sklearn\base.py:1389: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)


In [158]:
# LEFT_ARM
X_train4 = pd.read_csv("pose_data_csv/LEFT_ARM/train/X_train.csv").values
y_train4 = pd.read_csv("pose_data_csv/LEFT_ARM/train/y_train.csv").values
X_test4 = pd.read_csv("pose_data_csv/LEFT_ARM/test/X_test.csv").values
y_test4 = pd.read_csv("pose_data_csv/LEFT_ARM/test/y_test.csv").values

# Logistic Regression
model10 = LogisticRegression()
model10.fit(X_train4, y_train4)
y_pred10 = model10.predict(X_test4)
accuracy10 = accuracy_score(y_test4, y_pred10)
print(f"LEFT_ARM - Logistic Regression: {accuracy10}")

# Support Vector Machine
model11 = SVC()
model11.fit(X_train4, y_train4)
y_pred11 = model11.predict(X_test4)
accuracy11 = accuracy_score(y_test4, y_pred11)
print(f"LEFT_ARM - Support Vector Machine: {accuracy11}")

# Random Forest
model12 = RandomForestClassifier()
model12.fit(X_train4, y_train4)
y_pred12 = model12.predict(X_test4)
accuracy12 = accuracy_score(y_test4, y_pred12)
print(f"LEFT_ARM - Random Forest: {accuracy12}")

LEFT_ARM - Logistic Regression: 1.0
LEFT_ARM - Support Vector Machine: 1.0
LEFT_ARM - Random Forest: 1.0


d:\enviroments\envs\pytorch\lib\site-packages\sklearn\utils\validation.py:1408: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
d:\enviroments\envs\pytorch\lib\site-packages\sklearn\utils\validation.py:1408: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
d:\enviroments\envs\pytorch\lib\site-packages\sklearn\base.py:1389: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)


In [162]:
# RIGHT_LEG
X_train5 = pd.read_csv("pose_data_csv/RIGHT_LEG/train/X_train.csv").values
y_train5 = pd.read_csv("pose_data_csv/RIGHT_LEG/train/y_train.csv").values
X_test5 = pd.read_csv("pose_data_csv/RIGHT_LEG/test/X_test.csv").values
y_test5 = pd.read_csv("pose_data_csv/RIGHT_LEG/test/y_test.csv").values

# Logistic Regression
model13 = LogisticRegression()
model13.fit(X_train5, y_train5)
y_pred13 = model13.predict(X_test5)
accuracy13 = accuracy_score(y_test5, y_pred13)
print(f"RIGHT_LEG - Logistic Regression: {accuracy13}")

# Support Vector Machine
model14 = SVC()
model14.fit(X_train5, y_train5)
y_pred14 = model14.predict(X_test5)
accuracy14 = accuracy_score(y_test5, y_pred14)
print(f"RIGHT_LEG - Support Vector Machine: {accuracy14}")

# Random Forest
model15 = RandomForestClassifier()
model15.fit(X_train5, y_train5)
y_pred15 = model15.predict(X_test5)
accuracy15 = accuracy_score(y_test5, y_pred15)
print(f"RIGHT_LEG - Random Forest: {accuracy15}")

RIGHT_LEG - Logistic Regression: 1.0
RIGHT_LEG - Support Vector Machine: 1.0
RIGHT_LEG - Random Forest: 1.0


d:\enviroments\envs\pytorch\lib\site-packages\sklearn\utils\validation.py:1408: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
d:\enviroments\envs\pytorch\lib\site-packages\sklearn\utils\validation.py:1408: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
d:\enviroments\envs\pytorch\lib\site-packages\sklearn\base.py:1389: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)


In [164]:
# LEFT_LEG
X_train6 = pd.read_csv("pose_data_csv/LEFT_LEG/train/X_train.csv").values
y_train6 = pd.read_csv("pose_data_csv/LEFT_LEG/train/y_train.csv").values
X_test6 = pd.read_csv("pose_data_csv/LEFT_LEG/test/X_test.csv").values
y_test6 = pd.read_csv("pose_data_csv/LEFT_LEG/test/y_test.csv").values

# Logistic Regression
model16 = LogisticRegression()
model16.fit(X_train6, y_train6)
y_pred16 = model16.predict(X_test6)
accuracy16 = accuracy_score(y_test6, y_pred16)
print(f"LEFT_LEG - Logistic Regression: {accuracy16}")

# Support Vector Machine
model17 = SVC()
model17.fit(X_train6, y_train6)
y_pred17 = model17.predict(X_test6)
accuracy17 = accuracy_score(y_test6, y_pred17)
print(f"LEFT_LEG - Support Vector Machine: {accuracy17}")

# Random Forest
model18 = RandomForestClassifier()
model18.fit(X_train6, y_train6)
y_pred18 = model18.predict(X_test6)
accuracy18 = accuracy_score(y_test6, y_pred18)
print(f"LEFT_LEG - Random Forest: {accuracy18}")

LEFT_LEG - Logistic Regression: 1.0
LEFT_LEG - Support Vector Machine: 1.0
LEFT_LEG - Random Forest: 1.0


d:\enviroments\envs\pytorch\lib\site-packages\sklearn\utils\validation.py:1408: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
d:\enviroments\envs\pytorch\lib\site-packages\sklearn\utils\validation.py:1408: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
d:\enviroments\envs\pytorch\lib\site-packages\sklearn\base.py:1389: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)


In [165]:
# chọn mô hình tốt nhất để dự đoán
# chọn mô hình có độ chính xác cao nhất
# BODY
if accuracy1 > accuracy2 and accuracy1 > accuracy3:
    print("BODY - Logistic Regression")
elif accuracy2 > accuracy1 and accuracy2 > accuracy3:
    print("BODY - Support Vector Machine")
else:
    print("BODY - Random Forest")
    
# FACE_NECK
if accuracy4 > accuracy5 and accuracy4 > accuracy6:
    print("FACE_NECK - Logistic Regression")
elif accuracy5 > accuracy4 and accuracy5 > accuracy6:
    print("FACE_NECK - Support Vector Machine")
else:
    print("FACE_NECK - Random Forest")
    
# RIGHT_ARM
if accuracy7 > accuracy8 and accuracy7 > accuracy9:
    print("RIGHT_ARM - Logistic Regression")
elif accuracy8 > accuracy7 and accuracy8 > accuracy9:
    print("RIGHT_ARM - Support Vector Machine")
else:
    print("RIGHT_ARM - Random Forest")
    
# LEFT_ARM
if accuracy10 > accuracy11 and accuracy10 > accuracy12:
    print("LEFT_ARM - Logistic Regression")
elif accuracy11 > accuracy10 and accuracy11 > accuracy12:
    print("LEFT_ARM - Support Vector Machine")
else:
    print("LEFT_ARM - Random Forest")
    
# RIGHT_LEG
if accuracy13 > accuracy14 and accuracy13 > accuracy15:
    print("RIGHT_LEG - Logistic Regression")
elif accuracy14 > accuracy13 and accuracy14 > accuracy15:
    print("RIGHT_LEG - Support Vector Machine")
else:
    print("RIGHT_LEG - Random Forest")
    
# LEFT_LEG
if accuracy16 > accuracy17 and accuracy16 > accuracy18:
    print("LEFT_LEG - Logistic Regression")
elif accuracy17 > accuracy16 and accuracy17 > accuracy18:
    print("LEFT_LEG - Support Vector Machine")
else:
    print("LEFT_LEG - Random Forest")

BODY - Random Forest
FACE_NECK - Random Forest
RIGHT_ARM - Random Forest
LEFT_ARM - Random Forest
RIGHT_LEG - Random Forest
LEFT_LEG - Random Forest


In [166]:
# test mô hình bằng bộ tọa độ mới
# với bộ body thì nếu mô hình dự đoán là 1 thì trả ra 18, nếu dự đoán là 0 thì trả ra 0
# với bộ face_neck thì nếu mô hình dự đoán là 1 thì trả ra 9, nếu dự đoán là 0 thì trả ra 0
# với bộ right_arm thì nếu mô hình dự đoán là 1 thì trả ra 18, nếu dự đoán là 0 thì trả ra 0
# với bộ left_arm thì nếu mô hình dự đoán là 1 thì trả ra 18, nếu dự đoán là 0 thì trả ra 0
# với bộ right_leg thì nếu mô hình dự đoán là 1 thì trả ra 18, nếu dự đoán là 0 thì trả ra 0
# với bộ left_leg thì nếu mô hình dự đoán là 1 thì trả ra 18, nếu dự đoán là 0 thì trả ra 0
# sau đó cộng tổng lại sẽ ra được điểm số tổng

test_body = pd.read_csv("pose_data_csv/BODY/test/X_test.csv")
test_face_neck = pd.read_csv("pose_data_csv/FACE_NECK/test/X_test.csv")
test_right_arm = pd.read_csv("pose_data_csv/RIGHT_ARM/test/X_test.csv")
test_left_arm = pd.read_csv("pose_data_csv/LEFT_ARM/test/X_test.csv")
test_right_leg = pd.read_csv("pose_data_csv/RIGHT_LEG/test/X_test.csv")
test_left_leg = pd.read_csv("pose_data_csv/LEFT_LEG/test/X_test.csv")

In [168]:
# thử dự đoán với bộ test của từng bộ phận cơ thể
# BODY
y_pred1 = model1.predict(test_body)
print(f"BODY: {y_pred1}")
y_pred1 = [18 if i == 1 else 0 for i in y_pred1]
print(f"BODY: {y_pred1}")

# FACE_NECK
y_pred2 = model4.predict(test_face_neck)
print(f"FACE_NECK: {y_pred2}")
y_pred2 = [9 if i == 1 else 0 for i in y_pred2]
print(f"FACE_NECK: {y_pred2}")

# RIGHT_ARM
y_pred3 = model7.predict(test_right_arm)
print(f"RIGHT_ARM: {y_pred3}")
y_pred3 = [18 if i == 1 else 0 for i in y_pred3]
print(f"RIGHT_ARM: {y_pred3}")

# LEFT_ARM
y_pred4 = model10.predict(test_left_arm)
print(f"LEFT_ARM: {y_pred4}")
y_pred4 = [18 if i == 1 else 0 for i in y_pred4]
print(f"LEFT_ARM: {y_pred4}")

# RIGHT_LEG
y_pred5 = model13.predict(test_right_leg)
print(f"RIGHT_LEG: {y_pred5}")
y_pred5 = [18 if i == 1 else 0 for i in y_pred5]
print(f"RIGHT_LEG: {y_pred5}")

# LEFT_LEG
y_pred6 = model16.predict(test_left_leg)
print(f"LEFT_LEG: {y_pred6}")
y_pred6 = [18 if i == 1 else 0 for i in y_pred6]
print(f"LEFT_LEG: {y_pred6}")

# cộng tổng điểm số, output cũng là 1 list giống như y_pred1, y_pred2, ..., y_pred6
total_score = [sum(i) for i in zip(y_pred1, y_pred2, y_pred3, y_pred4, y_pred5, y_pred6)]
print(f"Total score: {total_score}")

BODY: [1 0 0 1 0 1 1 0 1 0 0 1 1 0 1 0 1 0 0 1 0 1 1 0 1 0 0 0 1 1 1 0 0 0 0 0 0
 0 0 1 0 0 0 0 1 1 1 0 0 0 0 0 1 0 0 0 0 1]
BODY: [18, 0, 0, 18, 0, 18, 18, 0, 18, 0, 0, 18, 18, 0, 18, 0, 18, 0, 0, 18, 0, 18, 18, 0, 18, 0, 0, 0, 18, 18, 18, 0, 0, 0, 0, 0, 0, 0, 0, 18, 0, 0, 0, 0, 18, 18, 18, 0, 0, 0, 0, 0, 18, 0, 0, 0, 0, 18]
FACE_NECK: [1 0 0 1 0 1 1 0 1 0 0 1 1 0 1 0 1 0 0 1 0 1 1 0 1 0 0 0 1 1 1 0 0 0 0 0 0
 0 0 1 0 0 0 0 1 1 1 0 0 0 0 0 1 0 0 0 0 1]
FACE_NECK: [9, 0, 0, 9, 0, 9, 9, 0, 9, 0, 0, 9, 9, 0, 9, 0, 9, 0, 0, 9, 0, 9, 9, 0, 9, 0, 0, 0, 9, 9, 9, 0, 0, 0, 0, 0, 0, 0, 0, 9, 0, 0, 0, 0, 9, 9, 9, 0, 0, 0, 0, 0, 9, 0, 0, 0, 0, 9]
RIGHT_ARM: [1 0 0 1 0 1 1 0 1 0 0 1 1 0 1 0 1 0 0 1 0 1 1 0 1 0 0 0 1 1 1 0 0 0 0 0 0
 0 0 1 0 0 0 0 1 1 1 0 0 0 0 0 1 0 0 0 0 1]
RIGHT_ARM: [18, 0, 0, 18, 0, 18, 18, 0, 18, 0, 0, 18, 18, 0, 18, 0, 18, 0, 0, 18, 0, 18, 18, 0, 18, 0, 0, 0, 18, 18, 18, 0, 0, 0, 0, 0, 0, 0, 0, 18, 0, 0, 0, 0, 18, 18, 18, 0, 0, 0, 0, 0, 18, 0, 0, 0, 0, 18]
LEFT_ARM: [1 0 0 1

d:\enviroments\envs\pytorch\lib\site-packages\sklearn\utils\validation.py:2732: UserWarning: X has feature names, but LogisticRegression was fitted without feature names
  warnings.warn(
d:\enviroments\envs\pytorch\lib\site-packages\sklearn\utils\validation.py:2732: UserWarning: X has feature names, but LogisticRegression was fitted without feature names
  warnings.warn(
d:\enviroments\envs\pytorch\lib\site-packages\sklearn\utils\validation.py:2732: UserWarning: X has feature names, but LogisticRegression was fitted without feature names
  warnings.warn(
d:\enviroments\envs\pytorch\lib\site-packages\sklearn\utils\validation.py:2732: UserWarning: X has feature names, but LogisticRegression was fitted without feature names
  warnings.warn(
d:\enviroments\envs\pytorch\lib\site-packages\sklearn\utils\validation.py:2732: UserWarning: X has feature names, but LogisticRegression was fitted without feature names
  warnings.warn(
d:\enviroments\envs\pytorch\lib\site-packages\sklearn\utils\valid

In [121]:
df = pd.read_csv("pose_data_csv/BODY/test/y_test.csv", header=None)

df.value_counts()

0    
0        36
1        21
label     1
Name: count, dtype: int64

In [ ]:
import matplotlib.pyplot as plt
import cv2 

# đọc video
cap = cv2.VideoCapture("test1.mp4")

# cắt frame